In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import numpy as np
import importlib
import nnwosd as wosd
importlib.reload(wosd)
from sfm_mle import estimate
import pandas as pd


# Number of Monte Carlo iterations
n_iter = 100
n_samples = 200

# True parameters
b_true = 2
w_true = torch.tensor([.2])
noise_std_v = 0.9
noise_std_u = 0.2

# Activations
activations = {
    "ReLU": nn.ReLU(),
    "ELU": nn.ELU(),
    "FlippedReLU": wosd.FlippedLeakRELU(alpha=0.8),
    "FlippedELU": wosd.FlippedELU(alpha=0.8)
}
clamp_activations = ["FlippedReLU", "FlippedELU"]

# Store results
RMSE_nn_all   = {k: [] for k in activations.keys()}
BIAS_nn_all   = {k: [] for k in activations.keys()}
bias_v_nn_all = {k: [] for k in activations.keys()}
bias_u_nn_all = {k: [] for k in activations.keys()}
Bias_TE_nn_all= {k: [] for k in activations.keys()}


# To store simulated datasets for each iteration
simulated_datasets = []

# Add SFM placeholders
RMSE_nn_all['sfm']   = []
BIAS_nn_all['sfm']   = []
bias_v_nn_all['sfm'] = []
bias_u_nn_all['sfm'] = []
Bias_TE_nn_all['sfm']= []

# Monte Carlo loop
for run in range(n_iter):
    print(f"\n--- Simulation run {run+1}/{n_iter} ---")

    # Generate data
    noise_v = torch.from_numpy(np.random.normal(0, noise_std_v, size=(n_samples, 1)).astype(np.float32))
    noise_u = torch.from_numpy(np.abs(np.random.normal(0, noise_std_u, size=(n_samples, 1)).astype(np.float32)))
    # X = torch.from_numpy((np.random.uniform(0.01, 3, size=(n_samples, 1)).astype(np.float32)))
    log_X = torch.from_numpy((np.random.uniform(-3, 3, size=(n_samples, 1)).astype(np.float32)))
    X = torch.exp(log_X)
    log_y = w_true * log_X + np.log(b_true) + (noise_v - noise_u)
    
    # Create a DataFrame with X, y, and noise terms
    sim_df = pd.DataFrame({
        "log_X": log_X.numpy().flatten(),
        "log_y": log_y.numpy().flatten()
    })
    
    simulated_datasets.append(sim_df)
    
    # Fit SFM (MLE)
    y = log_y.numpy().flatten()
    x1 = log_X.numpy().flatten()
    coefs, sterr, logMLE = estimate(y, x1, b_true, w_true, noise_std_u, noise_std_v)

    y_tf = ((X**w_true)*b_true).numpy()
    # y_sfm_mean = ((X**coefs[1])*np.exp(coefs[0]))
    #calculate the fitted y
    # e_sfm = log_y - (coefs[0] + coefs[1] * log_X)
    # e_sfm_np = e_sfm.detach().numpy()   # detach from graph, convert to NumPy
    # E_sfm_correction = np.mean(np.exp(e_sfm_np))
    E_sfm_correction = np.exp((coefs[2]+coefs[3])/2)
    
    y_sfm_mean = ((X.numpy() ** coefs[1]) * np.exp(coefs[0])) * E_sfm_correction
    
    RMSE_sfm = np.mean(((y_sfm_mean - y_tf) / y_tf)**2)
    BIAS_sfm = np.mean(np.abs((y_sfm_mean - y_tf) / y_tf))
    sigma_v_sfm = np.sqrt(coefs[3])
    sigma_u_sfm = np.sqrt(coefs[2])
    bias_v_sfm = np.abs(sigma_v_sfm - noise_std_v)
    bias_u_sfm = np.abs(sigma_u_sfm - noise_std_u)

    vectorized_TE_fun = np.vectorize(wosd.TE_fun)
    TE_true = vectorized_TE_fun(
        residuals=((np.log(y_tf)-(w_true * log_X + np.log(b_true)).numpy())),
        sig_v=noise_std_v, sig_u=noise_std_u
    )
    TE_sfm = vectorized_TE_fun(
        residuals=(np.log(y_sfm_mean) - log_y.numpy()),
        sig_v=sigma_v_sfm, sig_u=sigma_u_sfm
    )
    Bias_TE_sfm = np.mean(np.abs(TE_sfm - TE_true))

    # Store SFM results
    RMSE_nn_all['sfm'].append(float(RMSE_sfm))
    BIAS_nn_all['sfm'].append(float(BIAS_sfm))
    bias_v_nn_all['sfm'].append(float(bias_v_sfm))
    bias_u_nn_all['sfm'].append(float(bias_u_sfm))
    Bias_TE_nn_all['sfm'].append(float(Bias_TE_sfm))

    # Standardize data for NN
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X_standardized = torch.tensor(scaler_X.fit_transform(log_X), dtype=torch.float32)
    y_standardized = torch.tensor(scaler_y.fit_transform(log_y), dtype=torch.float32)

    # Train NN models
    for name, activation_fun in activations.items():
        model = wosd.MLP(1, [32, 8], 1, activation_func=activation_fun)
        nll_loss = wosd.GaussianNLLLoss(sigma_v=noise_std_v, sigma_u=noise_std_u)
        optimizer = optim.Adam(list(model.parameters()) + [nll_loss.log_std_v, nll_loss.log_std_u], lr=0.01)

        best_loss = float('inf')
        best_state = None
        for epoch in range(1000):
            model.train()
            y_pred = model(X_standardized)
            loss = nll_loss(y_pred, y_standardized)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if name in clamp_activations:
                with torch.no_grad():
                    for layer in model.layers:
                        layer.weight.data.clamp_(min=0)
                    model.output.weight.clamp_(min=0)

            if loss.item() < best_loss:
                best_loss = loss.item()
                best_state = model.state_dict()

        model.load_state_dict(best_state)

        sigma_v_nn = torch.exp(nll_loss.log_std_v).item() * scaler_y.scale_
        sigma_u_nn = torch.exp(nll_loss.log_std_u).item() * scaler_y.scale_

        with torch.no_grad():
            y_pred_std = model(X_standardized)
            y_pred_std = y_pred_std + np.sqrt(2/np.pi) * sigma_u_nn
            y_pred_original = scaler_y.inverse_transform(y_pred_std.numpy())
            #calculate the fitted value
            # y_original = scaler_y.inverse_transform(y_standardized.numpy())
            # residuals_nn= y_original - y_pred_original
            # E_nn_correction = residuals_nn.mean()
            E_nn_correction = np.exp((sigma_v_nn**2+sigma_u_nn**2)/2)
            y_pred_original = np.exp(y_pred_original)*E_nn_correction

        rmse = np.sqrt(np.mean(((y_pred_original - y_tf) / y_tf)**2))
        bias = np.mean(np.abs(y_pred_original - y_tf) / y_tf)
        bias_v = np.abs(sigma_v_nn - noise_std_v)
        bias_u = np.abs(sigma_u_nn - noise_std_u)

        TE_nn = vectorized_TE_fun(
            residuals=(y_pred_original - log_y.numpy()),
            sig_v=sigma_v_nn, sig_u=sigma_u_nn
        )
        te_bias = np.mean(np.abs(TE_nn - TE_true))

        
        # Store results
        RMSE_nn_all[name].append(float(rmse))
        BIAS_nn_all[name].append(float(bias))
        bias_v_nn_all[name].append(float(bias_v))
        bias_u_nn_all[name].append(float(bias_u))
        Bias_TE_nn_all[name].append(float(te_bias))
    
    
# Summary statistics
def summarize_results(metric_dict):
    return {k: (np.mean(v), np.std(v)) for k, v in metric_dict.items()}

print("\n=== Mean (std) RMSE over 100 runs ===")
print(summarize_results(RMSE_nn_all))
print("\n=== Mean (std) BIAS over 100 runs ===")
print(summarize_results(BIAS_nn_all))
print("\n=== Mean (std) bias_v over 100 runs ===")
print(summarize_results(bias_v_nn_all))
print("\n=== Mean (std) bias_u over 100 runs ===")
print(summarize_results(bias_u_nn_all))
print("\n=== Mean (std) TE bias over 100 runs ===")
print(summarize_results(Bias_TE_nn_all))



--- Simulation run 1/100 ---

--- Simulation run 2/100 ---

--- Simulation run 3/100 ---

--- Simulation run 4/100 ---

--- Simulation run 5/100 ---

--- Simulation run 6/100 ---

--- Simulation run 7/100 ---

--- Simulation run 8/100 ---

--- Simulation run 9/100 ---

--- Simulation run 10/100 ---


/home/zwei1/NNSFM/good_to_go/New_Sim_Dog_Cobbs/sfm_mle.py:26: RuntimeWarning: divide by zero encountered in log
  logDen = np.log(Den)
/home/zwei1/.conda/envs/d2l/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0



--- Simulation run 11/100 ---

--- Simulation run 12/100 ---

--- Simulation run 13/100 ---

--- Simulation run 14/100 ---

--- Simulation run 15/100 ---

--- Simulation run 16/100 ---

--- Simulation run 17/100 ---


/home/zwei1/NNSFM/good_to_go/New_Sim_Dog_Cobbs/sfm_mle.py:26: RuntimeWarning: divide by zero encountered in log
  logDen = np.log(Den)
/home/zwei1/.conda/envs/d2l/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0



--- Simulation run 18/100 ---


...

/home/zwei1/NNSFM/good_to_go/New_Sim_Dog_Cobbs/sfm_mle.py:26: RuntimeWarning: divide by zero encountered in log
  logDen = np.log(Den)
/home/zwei1/.conda/envs/d2l/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0



--- Simulation run 94/100 ---

--- Simulation run 95/100 ---


/home/zwei1/NNSFM/good_to_go/New_Sim_Dog_Cobbs/sfm_mle.py:26: RuntimeWarning: divide by zero encountered in log
  logDen = np.log(Den)
/home/zwei1/.conda/envs/d2l/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0



--- Simulation run 96/100 ---

--- Simulation run 97/100 ---

--- Simulation run 98/100 ---

--- Simulation run 99/100 ---


/home/zwei1/NNSFM/good_to_go/New_Sim_Dog_Cobbs/sfm_mle.py:26: RuntimeWarning: divide by zero encountered in log
  logDen = np.log(Den)
/home/zwei1/.conda/envs/d2l/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


In [1]:
def summarize_results(metric_dict):
    return {k: (np.mean(v), np.std(v)) for k, v in metric_dict.items()}

print("\n=== Mean (std) RMSE over 100 runs ===")
print(summarize_results(RMSE_nn_all))
print("\n=== Mean (std) BIAS over 100 runs ===")
print(summarize_results(BIAS_nn_all))
print("\n=== Mean (std) bias_v over 100 runs ===")
print(summarize_results(bias_v_nn_all))
print("\n=== Mean (std) bias_u over 100 runs ===")
print(summarize_results(bias_u_nn_all))
print("\n=== Mean (std) TE bias over 100 runs ===")
print(summarize_results(Bias_TE_nn_all))



=== Mean (std) RMSE over 100 runs ===


NameError: name 'RMSE_nn_all' is not defined

In [ ]:
# simulated_datasets

In [ ]:
import os

# Create output directory
os.makedirs("simulated_data", exist_ok=True)

for i, sim_df in enumerate(simulated_datasets, start=1):
    file_path = f"simulated_data/sim_data_iter_{i}.csv"
    sim_df.to_csv(file_path, index=False)

print(f"Saved {len(simulated_datasets)} simulated datasets in 'simulated_data/'")

In [ ]:
# import pickle
# import os

# os.makedirs("simulated_data", exist_ok=True)

# # Save the simulated_datasets list
# with open("simulated_data/simulated_datasets.pkl", "wb") as f:
#     pickle.dump(simulated_datasets, f)

# # Save the metrics as a dict
# metrics_dict = {
#     "RMSE_nn_all": RMSE_nn_all,
#     "BIAS_nn_all": BIAS_nn_all,
#     "bias_v_nn_all": bias_v_nn_all,
#     "bias_u_nn_all": bias_u_nn_all,
#     "Bias_TE_nn_all": Bias_TE_nn_all
# }
# with open("simulated_data/metrics.pkl", "wb") as f:
#     pickle.dump(metrics_dict, f)

# print("Saved simulated datasets and metrics as Pickle files.")

In [7]:
import numpy as np
# Summary statistics
def summarize_results(metric_dict):
    return {k: (np.mean(v), np.std(v)) for k, v in metric_dict.items()}


In [8]:
import pickle

# Load simulated datasets
with open("simulated_data/simulated_datasets.pkl", "rb") as f:
    simulated_datasets = pickle.load(f)

# Load metrics
with open("simulated_data/metrics.pkl", "rb") as f:
    metrics = pickle.load(f)

print(metrics.keys())

dict_keys(['RMSE_nn_all', 'BIAS_nn_all', 'bias_v_nn_all', 'bias_u_nn_all', 'Bias_TE_nn_all'])


In [9]:
print("\n=== Mean (std) RMSE over 100 runs ===")
print(summarize_results(metrics['RMSE_nn_all']))
print("\n=== Mean (std) BIAS over 100 runs ===")
print(summarize_results(metrics['BIAS_nn_all']))
print("\n=== Mean (std) bias_v over 100 runs ===")
print(summarize_results(metrics['bias_v_nn_all']))
print("\n=== Mean (std) bias_u over 100 runs ===")
print(summarize_results(metrics['bias_u_nn_all']))
print("\n=== Mean (std) TE bias over 100 runs ===")
print(summarize_results(metrics['Bias_TE_nn_all']))


=== Mean (std) RMSE over 100 runs ===
{'ReLU': (0.6390168641938269, 0.1480993092579175), 'ELU': (0.5977172084726952, 0.14090949680232984), 'FlippedReLU': (0.5187590563422317, 0.13158599630693982), 'FlippedELU': (0.5236267867599146, 0.1312897730606343), 'sfm': (0.27425441712141035, 0.03524368139043235)}

=== Mean (std) BIAS over 100 runs ===
{'ReLU': (0.5082098294873404, 0.12250912906656748), 'ELU': (0.5102376253767004, 0.12505464429725813), 'FlippedReLU': (0.507530885288058, 0.1310673562661376), 'FlippedELU': (0.50844922941556, 0.1310495505635086), 'sfm': (0.5158164793252945, 0.029842520204196636)}

=== Mean (std) bias_v over 100 runs ===
{'ReLU': (0.05957000657729781, 0.03955621440945941), 'ELU': (0.04686149149640669, 0.035348941855314016), 'FlippedReLU': (0.039995576792693034, 0.028350121312007186), 'FlippedELU': (0.039975268061382165, 0.028891607042543118), 'sfm': (0.00883245475870044, 0.0177473529373095)}

=== Mean (std) bias_u over 100 runs ===
{'ReLU': (0.013237697500287663, 0.0